# Experiment 006: Find Better Complete Snapshot

Strategy: The ensemble approach is broken (5 failed submissions). We need to find a COMPLETE snapshot that:
1. Has a better score than baseline (70.676102)
2. Is used AS-IS without mixing configurations
3. Should pass Kaggle validation since it's from a known-good source

In [ ]:
import sys
sys.path.insert(0, '/home/code')

import pandas as pd
import numpy as np
import os
import glob
from utils import load_submission, score_submission, is_valid_submission
import json

print("Utilities loaded successfully!")

In [ ]:
# Find all CSV files in snapshots
snapshot_dir = '/home/nonroot/snapshots/santa-2025/'
all_csvs = glob.glob(f'{snapshot_dir}/**/*.csv', recursive=True)
print(f"Found {len(all_csvs)} CSV files in snapshots")

# Filter to valid submissions only
valid_submissions = []
for csv_path in all_csvs:
    try:
        df = pd.read_csv(csv_path)
        if is_valid_submission(df):
            valid_submissions.append(csv_path)
    except Exception as e:
        pass

print(f"Found {len(valid_submissions)} valid submission files")

In [ ]:
# Score each complete submission and find the best ones
scores = []

for i, csv_path in enumerate(valid_submissions):
    try:
        df = pd.read_csv(csv_path)
        score, _, _ = score_submission(df, check_overlaps=False)
        scores.append((score, csv_path))
        
        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1}/{len(valid_submissions)} submissions...")
    except Exception as e:
        pass

print(f"\nScored {len(scores)} submissions")

In [ ]:
# Sort by score (lower is better)
scores.sort(key=lambda x: x[0])

# Show top 20 best scores
print("Top 20 best complete submissions:")
print("-" * 80)
for i, (score, path) in enumerate(scores[:20]):
    short_path = '/'.join(path.split('/')[-3:])
    print(f"{i+1:2d}. Score: {score:.6f} - {short_path}")

print(f"\nBaseline score: 70.676102")
print(f"Best found: {scores[0][0]:.6f}")
print(f"Improvement over baseline: {70.676102 - scores[0][0]:.6f}")

In [ ]:
# Check if the best submission is different from our baseline
best_score, best_path = scores[0]
print(f"\nBest submission: {best_path}")
print(f"Score: {best_score:.6f}")

# Load and compare with baseline
baseline_path = '/home/code/experiments/000_baseline/submission.csv'
baseline_df = pd.read_csv(baseline_path)
best_df = pd.read_csv(best_path)

# Check if they're the same
if baseline_df.equals(best_df):
    print("\nBest submission is IDENTICAL to baseline.")
else:
    print("\nBest submission is DIFFERENT from baseline!")
    # Check how many rows differ
    diff_count = (baseline_df != best_df).sum().sum()
    print(f"Number of differing values: {diff_count}")

In [ ]:
# If best is different and better, use it
if best_score < 70.676102 - 0.0001:  # At least 0.0001 improvement
    print(f"\nFound better submission!")
    print(f"Score: {best_score:.6f} (improvement: {70.676102 - best_score:.6f})")
    
    # Copy to experiment folder and submission folder
    import shutil
    work_dir = '/home/code/experiments/006_find_better_snapshot'
    
    shutil.copy(best_path, f'{work_dir}/submission.csv')
    shutil.copy(best_path, '/home/submission/submission.csv')
    print(f"Copied to {work_dir}/submission.csv and /home/submission/submission.csv")
    
    # Save metrics
    metrics = {
        'cv_score': best_score,
        'baseline_score': 70.676102,
        'improvement': 70.676102 - best_score,
        'source_path': best_path
    }
    with open(f'{work_dir}/metrics.json', 'w') as f:
        json.dump(metrics, f, indent=2)
    print(f"Metrics saved: {metrics}")
else:
    print(f"\nNo better complete submission found.")
    print(f"Best score: {best_score:.6f}")
    print(f"Baseline score: 70.676102")
    
    # Use baseline as fallback
    import shutil
    work_dir = '/home/code/experiments/006_find_better_snapshot'
    shutil.copy(baseline_path, f'{work_dir}/submission.csv')
    shutil.copy(baseline_path, '/home/submission/submission.csv')
    
    metrics = {
        'cv_score': 70.676102,
        'baseline_score': 70.676102,
        'improvement': 0.0,
        'notes': 'No better complete snapshot found, using baseline'
    }
    with open(f'{work_dir}/metrics.json', 'w') as f:
        json.dump(metrics, f, indent=2)
    print(f"Using baseline as fallback.")

In [ ]:
# Summary
print("=" * 60)
print("EXPERIMENT 006: FIND BETTER SNAPSHOT SUMMARY")
print("=" * 60)
print(f"Total submissions scanned: {len(valid_submissions)}")
print(f"Baseline score: 70.676102")
print(f"Best complete snapshot: {best_score:.6f}")
print(f"Improvement: {70.676102 - best_score:.6f}")
print(f"\nTarget: 68.888293")
print(f"Gap to target: {best_score - 68.888293:.6f} ({(best_score - 68.888293) / 68.888293 * 100:.2f}%)")
print("=" * 60)